In [1]:
import polars as pl

In [2]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [10]:

health_survey_df = pl.read_csv('./Data/health_survey.csv')
reverse_coding_df = pl.read_csv('./Data/ReverseCodingItems.csv')



In [3]:
import polars as pl
health_survey_df = pl.read_csv('./Data/health_survey.csv')
reverse_coding_df = pl.read_csv('./Data/ReverseCodingItems.csv')
health_survey_df = health_survey_df.with_columns(
    pl.Series(name="Respondent_ID", values=range(1, health_survey_df.height + 1))
)
stacked_survey = health_survey_df.unpivot(
    index=["Respondent_ID"], 
    on=[col for col in health_survey_df.columns if col != "Respondent_ID"], 
    variable_name="Question", 
    value_name="Response"
)
merged_survey = stacked_survey.join(reverse_coding_df, on="Question", how="left")
merged_survey = merged_survey.with_columns(
    (pl.col("Needs Reverse Coding?") == "Yes").alias("Needs Reverse")
)

coding_map = {
    "Strongly Disagree": 1,
    "Somewhat Disagree": 2,
    "Neither Agree nor Disagree": 3,
    "Somewhat Agree": 4,
    "Strongly Agree": 5
}

reverse_coding_map = {
    "Strongly Disagree": 5,
    "Somewhat Disagree": 4,
    "Neither Agree nor Disagree": 3,
    "Somewhat Agree": 2,
    "Strongly Agree": 1
}


merged_survey = merged_survey.with_columns([
    pl.when(pl.col("Response") == "Strongly Disagree").then(1)
      .when(pl.col("Response") == "Somewhat Disagree").then(2)
      .when(pl.col("Response") == "Neither Agree nor Disagree").then(3)
      .when(pl.col("Response") == "Somewhat Agree").then(4)
      .when(pl.col("Response") == "Strongly Agree").then(5)
      .alias("Temp Coded Value"),

    pl.when(pl.col("Response") == "Strongly Disagree").then(5)
      .when(pl.col("Response") == "Somewhat Disagree").then(4)
      .when(pl.col("Response") == "Neither Agree nor Disagree").then(3)
      .when(pl.col("Response") == "Somewhat Agree").then(2)
      .when(pl.col("Response") == "Strongly Agree").then(1)
      .alias("Temp Coded Value (Reversed)")
])


merged_survey = merged_survey.with_columns(
    pl.when(pl.col("Needs Reverse"))
      .then(pl.col("Temp Coded Value (Reversed)"))
      .otherwise(pl.col("Temp Coded Value"))
      .alias("Recoded Value")
)

merged_survey = merged_survey.with_columns(
    pl.format("F{}", pl.col("Question")).alias("Question Type")
)

aggregated_data = merged_survey.group_by(["Respondent_ID", "Question Type"]).agg(
    pl.col("Recoded Value").mean()
)

aggregated_data.head()
aggregated_data.write_csv('/mnt/data/health_survey_summary.csv')

Respondent_ID,Question Type,Recoded Value
i64,str,f64
184,"""FF6""",2.0
208,"""FF2.3""",5.0
55,"""FF1.6""",4.0
157,"""FF2.10""",3.0
212,"""FF3.2""",3.0
